# ELE435/535 - LAB 9

## Main Parts of the LAB:
## 1) Kernel-PCA on synthetic data
## 2) Kernel-SVM and Kernel-RR on MNIST
## 3) Kernel-Nearest Neighbor on Liver Disorders Dataset
****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************


## 1) Kernel-PCA

In this assignment, we will study the effectiveness of kernel PCA.

1-1) Import the given data "LAB9_data_PCA.npy" into numpy. It contains 1000 points, the first 500 of which belongs to class #1 and the rest belongs to class #2. Each row corresponds to one data point.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sklearn
from time import time
import datetime
%matplotlib inline

1-2) Visualize the data in 3D. Use different colors for objects from distinct classes.

1-3) Now perform a normal PCA (i.e. linear PCA) over the data (you may use sklearn's built-in commands for this part). Then project the data onto the span of the first two principal components. Visualize the projected data in 2D. 

1-4) Now, perform kernel PCA with a 4-th order inhomogeneous polynomial kernel. Then, project the data onto the span of first two principal components and visualize the projected data in 2D **(do not use sklearn's built-in commands for this part; follow the steps mentioned in the notes, e.g. find the Gram matrix of the centered data, then use SVD, etc.)**

1-5) Implement kernel PCA with the RBF kernel **(do not use sklearn's built-in commands; follow the steps in the notes--similar to what you did in the previous part) **
$(k(x,z) = e^{-\gamma \lVert {x-z} \lVert_2^2}, \gamma = 1/2\sigma^2)$.
Choose sigma to be 10, 30, and 100. Also, visualize the projected data onto the span of the first two principal components for each value of sigma.

1-6) How can you estimate a suitable value of sigma given a dataset? i.e., give a metric to estimate an appropriate value of sigma given some data points.  

## 2) Kernel-SVM and Kernel-RR on MNIST

In this part, we will use kernel-SVM and kernel-RR to classify MNIST digits. 


2-1) First, import the subset of MNIST dataset that we were working on in the previous labs. 

2-2) Report the classification accuracy on the testing data using kernel-SVM with a 'linear' kernel. (You may use sklearn's built-in commands for this part. If you use LinearSVC, make sure to change the loss argument to 'hinge', as unlike SVC, LinearSVC uses the squared hinge loss by default.)

2-3) Now, use SVM with 'rbf' kernel. Select three different values of 'gamma' and report how the test accuracy changes. (you may use sklearn's built-in commands for this part) Hint: In order to get a feeling for selecting an appropriate value for gamma, take a look at http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html.

2-4) Repeat 2-3 with the Kernel Ridge Classifier. You may use sklearn's KernelRidge, which is normally used for regression. To adopt it for classification, you can perform a binary encoding of the labels using LabeLBinarizer. Check https://scikit-learn.org/stable/modules/generated/sklearn.kernel_ridge.KernelRidge.html, https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelBinarizer.html.

# 3) Kernel Nearest Neighbor Classifier on Liver Disorders Data Set

In this part, we will be working with BUPA Liver Disorder database from BUPA Medical Research Ltd. (http://archive.ics.uci.edu/ml/datasets/Liver+Disorders). It contains data from 345 male individuals. We have selected the first 245 samples to form the training data and the rest as testing data. 

The first 5 features are results from blood tests. They are sensitive to liver disorders that may arise from excessive alcohol consumption. The sixth feature is the number of drinks per day. The last column specifies whether someone has the disorder or not.

3-1) Implement 1-nearest neighbor classifier and report the testing accuracy. You may use sklearn's built-in commands for this part.

3-2) Implement kernel-nearest neighbor classifier using 3-th order inhomogeneous polynomial kernel. Report the testing accuracy. **Do not use sklearn for this part. Follow the steps mentioned in the notes (i.e. through forming the Gram matrix).** How much test accuracy improvement do we achieve using the polynomial kernel?